1、普通attention 560

2、普通ESIM 569

3、加入full pretrain的ESIM 5779  加大deep到450 5796  HIDDEN_DIM = 200 DEEP_LAYERS = [500] 5811


In [1]:
import sys
import jieba
from gensim.models import Word2Vec
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR
from torchtext import data
from time import time
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# hyper parameters
STR_MAXLEN = 30
BATCH_SIZE = 256
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
EMBED_DIM = 300
ATTEN_DIM = 200
HIDDEN_DIM = 200
DEEP_LAYERS = [400]
LEARNING_RATE = 1e-3
EPOCHES = 20
DECAY_STEP = 2
DECAY_GAMMA = 0.99
CLASS_WEIGHT = [0.6116, 2.7397]
def print_flush(data, args=None):
    if args == None:
        print(data)
    else:
        print(data, args)
    sys.stdout.flush()
    
def get_mask(txt1, txt2):
    txt1 = txt1.cpu().numpy()
    txt2 = txt2.cpu().numpy()
    len_txt1 = len(txt1[0])
    len_txt2 = len(txt2[0])
    mask_txt1 = np.zeros(shape=[len(txt1), len_txt1])
    mask_txt2 = np.zeros(shape=[len(txt2), len_txt2])
    for i in range(len(txt1)):
        mask_len1 = np.where(txt1[i] == 1)[0]
        mask_len2 = np.where(txt2[i] == 1)[0]
        if len(mask_len1) == 0:
            mask_txt1[i, :] = 1
        else:
            mask_txt1[i, :mask_len1[0]] = 1
        if len(mask_len2) == 0:
            mask_txt2[i, :] = 1
        else:
            mask_txt2[i, :mask_len2[0]] = 1
    return torch.tensor(mask_txt1).float().to(DEVICE), torch.tensor(mask_txt2).float().to(DEVICE)

def wordmodel_to_matrix(pretrain_path, wordlist, device, dim=300):
    word_vec = Word2Vec.load(pretrain_path).wv
    word_vec_list = []
    oov = 0
    oov_words = []
    for idx, word in enumerate(wordlist):
        try:
            vector = np.array(word_vec[word], dtype=float).reshape(1,dim)
        except:
            oov += 1
            oov_words.append(word)
            # print(word)
            vector = np.random.rand(1, dim)
        word_vec_list.append(torch.from_numpy(vector))
    wordvec_matrix = torch.cat(word_vec_list)
    print("Load embedding finished.")
    print("Total words count: {}, oov count: {}.".format(wordvec_matrix.size()[0], oov))
    return wordvec_matrix if device == -1 else wordvec_matrix.to(device)

def wordlist_to_matrix(pretrain_path, wordlist, device, dim=300):
    word_vec = {}
    with open(pretrain_path, encoding='utf-8') as fr:
        for line in fr:
            line = line.split(' ')
            word = line[0]
            vec = line[1:]
            word_vec[word] = np.array(vec, dtype=float)
    word_vec_list = []
    oov = 0
    oov_words = []
    for idx, word in enumerate(wordlist):
        try:
            vector = np.array(word_vec[word], dtype=float).reshape(1,dim)
        except:
            oov += 1
            oov_words.append(word)
            # print(word)
            vector = np.random.rand(1, dim)
        word_vec_list.append(torch.from_numpy(vector))
    wordvec_matrix = torch.cat(word_vec_list)
    print("Load embedding finished.")
    print("Total words count: {}, oov count: {}.".format(wordvec_matrix.size()[0], oov))
    return wordvec_matrix if device == -1 else wordvec_matrix.to(device)

# torch.manual_seed(666)

D:\Anaconda\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
# prepare data

def tokenizer(text): # create a tokenizer function
    return [txt for txt in text]
class BatchWrapper:
    def __init__(self, dl, iter_columns):
        self.dl, self.iter_columns = dl, iter_columns  # we pass in the list of attributes for x &amp;amp;amp;amp;lt;g class="gr_ gr_3178 gr-alert gr_spell gr_inline_cards gr_disable_anim_appear ContextualSpelling ins-del" id="3178" data-gr-id="3178"&amp;amp;amp;amp;gt;and y&amp;amp;amp;amp;lt;/g&amp;amp;amp;amp;gt;

    def __iter__(self):
        for batch in self.dl:
            yield (getattr(batch, attr) for attr in self.iter_columns)

    def __len__(self):
        return len(self.dl)
    
print_flush('process raw data...')
TEXT = data.Field(sequential=True, use_vocab=True, eos_token='<EOS>', init_token='<BOS>',pad_token='<PAD>', 
                  batch_first=True, tokenize=tokenizer)
LABEL = data.Field(sequential=False, use_vocab=False, batch_first=True)

tv_datafields = [("id", None), # we won't be needing the id, so we pass in None as the field
                 ("txt1", TEXT), ("txt2", TEXT),
                 ("label", LABEL)]

train = data.TabularDataset(path='../datasets/train.csv', format='csv', skip_header=True, fields=tv_datafields)
valid = data.TabularDataset(path='../datasets/valid.csv', format='csv', skip_header=True, fields=tv_datafields)

TEXT.build_vocab(train, valid, min_freq=3)
print_flush('Building vocabulary Finished.')

# matrix = wordlist_to_matrix('../datasets/pretrain_embedding/pretrain_full_emb.txt', TEXT.vocab.itos, DEVICE)
matrix = wordmodel_to_matrix('../datasets/pretrain_embedding/raw_embedding_300d.bin', TEXT.vocab.itos, DEVICE)

process raw data...
Building vocabulary Finished.
Load embedding finished.
Total words count: 1517, oov count: 216.


In [3]:
len(TEXT.vocab)

1517

In [4]:
train_iter = data.BucketIterator(dataset=train, batch_size=BATCH_SIZE, sort_key=lambda x: len(x.txt1) + len(x.txt2), shuffle=False, device=-1, repeat=False)
valid_iter = data.Iterator(dataset=valid, batch_size=BATCH_SIZE, device=-1, shuffle=False, repeat=False)

train_dl = BatchWrapper(train_iter, ["txt1", "txt2", "label"])
valid_dl = BatchWrapper(valid_iter, ["txt1", "txt2", "label"])

print_flush('prepare data done!')

prepare data done!


In [5]:
def cosine_similarity(x1, x2):
    '''compute cosine similarity between x1 and x2
    Parameters
    ----------
    x1, x2 : 2-D torch Tensor
        size (batch_size, 1)
    Returns
    -------
    distance : 2-D torch Tensor
        similarity result of size (batch_size, 1)
    '''
    return F.cosine_similarity(x1, x2).unsqueeze(1)
def attention_matrix(x1, x2, eps=1e-6):
    '''compute attention matrix using match score
    
    1 / (1 + |x · y|)
    |·| is euclidean distance
    Parameters
    ----------
    x1, x2 : 4-D torch Tensor
        size (batch_size, 1, sentence_length, h)
    
    Returns
    -------
    output : 3-D torch Tensor
        match score result of size (batch_size, sentence_length(for x2), sentence_length(for x1))
    '''
    eps = torch.tensor(eps).to(DEVICE)
    one = torch.tensor(1.).to(DEVICE)
    euclidean = (torch.pow(x1 - x2.permute(0, 2, 1, 3), 2).sum(dim=3) + eps).sqrt()
    return (euclidean + one).reciprocal()



In [6]:
def highway_network(highway_network_linear, data_in, num_layers, output_size = None):
    def highway_layer(highway_network_linear, data_in, output_size = None):
        size = data_in.size()
        if len(size) > 2:
            inputs = data_in.view(size[0]*size[1], size[2])
        else:
            inputs = data_in
        if output_size is not None:
            d = output_size
        else:
            d = data_in.size()[-1]
        trans = highway_network_linear(inputs)
        trans = F.relu(trans)
        gate = highway_network_linear(inputs)
        gate = F.sigmoid(gate)
        if d != data_in.size()[-1]:
            data_in = highway_network_linear(inputs)
            out = gate * trans + (1 - gate) * inputs
        else:
            out = gate * trans + (1 - gate) * inputs
        if len(size) > 2:
            out = out.view(size[0], size[1], size[2])
        return out

    prev = data_in
    for layer_idx in range(num_layers):
        cur = highway_layer(highway_network_linear, prev, output_size = output_size)
        prev = cur
    return cur

class CoAttention(nn.Module):
    def __init__(self, input_dim, atten_dim):
        super(CoAttention, self).__init__()
        self.input_dim = input_dim
        self.atten_dim = atten_dim
        self.attention_align = nn.Linear(input_dim, atten_dim)
        self.attention_max = nn.Linear(input_dim, atten_dim)
        self.attention_mean = nn.Linear(input_dim, atten_dim)
        self.attention_self = nn.Linear(input_dim, atten_dim)
        self.compress_align = Compression(input_dim, 'nn')
        self.compress_max = Compression(input_dim, 'nn')
        self.compress_mean = Compression(input_dim, 'nn')
        self.compress_self = Compression(input_dim, 'nn')
        
    def forward(self, lstm_txt1, lstm_txt2, x1_mask=None, x2_mask=None):
        # alignment pooling
        size1 = lstm_txt1.size()
        size2 = lstm_txt2.size()
        align_atten = torch.bmm(F.relu(self.attention_align(lstm_txt1.view(size1[0]*size1[1], size1[2]))).view(size1[0], size1[1], self.atten_dim),
                        F.relu(self.attention_align(lstm_txt2.view(size2[0]*size2[1], size2[2]))).view(size2[0], size2[1], self.atten_dim).transpose(1, 2))
        
        alphas = F.softmax(align_atten, 2)
        beta = F.softmax(align_atten, 1)
#         alphas = alphas * x1_mask[:, :, None]
#         beta = beta * x2_mask[:, None, :]
        align_txt1 = torch.bmm(alphas, lstm_txt2)
        align_txt2 = torch.bmm(beta.permute(0, 2, 1), lstm_txt1)
        
        fc_align_txt1 = torch.cat([align_txt1, lstm_txt1], 2)
        fm_align_txt1 = align_txt1 * lstm_txt1
        fs_align_txt1 = align_txt1 - lstm_txt1
        fc_align_txt2 = torch.cat([align_txt2, lstm_txt2], 2)
        fm_align_txt2 = align_txt2 * lstm_txt2
        fs_align_txt2 = align_txt2 - lstm_txt2
        
        fc_align_txt1, fm_align_txt1, fs_align_txt1 = self.compress_align(fc_align_txt1, fm_align_txt1, fs_align_txt1)
        fc_align_txt2, fm_align_txt2, fs_align_txt2 = self.compress_align(fc_align_txt2, fm_align_txt2, fs_align_txt2)
        
        # max pooling
        max_atten = torch.bmm(F.relu(self.attention_max(lstm_txt1.view(size1[0]*size1[1], size1[2]))).view(size1[0], size1[1], self.atten_dim),
                        F.relu(self.attention_max(lstm_txt2.view(size2[0]*size2[1], size2[2]))).view(size2[0], size2[1], self.atten_dim).transpose(1, 2))
        max_row = F.softmax(torch.max(max_atten, 2, True)[0], dim=1)
        max_row = max_row.repeat(1, 1, lstm_txt2.size(1))
        max_txt1 = torch.bmm(max_row, lstm_txt2)
        
        max_col = torch.max(max_atten, 1, True)[0]
        max_col = F.softmax(max_col, dim=2)
        max_col = max_col.repeat(1, lstm_txt1.size(1), 1).transpose(1, 2)
        max_txt2 = torch.bmm(max_col, lstm_txt1)
        
        fc_max_txt1 = torch.cat([max_txt1, lstm_txt1], 2)
        fm_max_txt1 = max_txt1 * lstm_txt1
        fs_max_txt1 = max_txt1 - lstm_txt1
        fc_max_txt2 = torch.cat([max_txt2, lstm_txt2], 2)
        fm_max_txt2 = max_txt2 * lstm_txt2
        fs_max_txt2 = max_txt2 - lstm_txt2
        
        fc_max_txt1, fm_max_txt1, fs_max_txt1 = self.compress_max(fc_max_txt1, fm_max_txt1, fs_max_txt1)
        fc_max_txt2, fm_max_txt2, fs_max_txt2 = self.compress_max(fc_max_txt2, fm_max_txt2, fs_max_txt2)
        
        # mean pooling
        mean_atten = torch.bmm(F.relu(self.attention_mean(lstm_txt1.view(size1[0]*size1[1], size1[2]))).view(size1[0], size1[1], self.atten_dim),
                        F.relu(self.attention_mean(lstm_txt2.view(size2[0]*size2[1], size2[2]))).view(size2[0], size2[1], self.atten_dim).transpose(1, 2))
        mean_row = F.softmax(torch.mean(mean_atten, 2, True), dim=1)
        mean_row = mean_row.repeat(1, 1, lstm_txt2.size(1))
        mean_txt1 = torch.bmm(mean_row, lstm_txt2)
        
        mean_col = F.softmax(torch.mean(mean_atten, 1, True), dim=2)
        mean_col = mean_col.repeat(1, lstm_txt1.size(1), 1).transpose(1, 2)
        mean_txt2 = torch.bmm(mean_col, lstm_txt1)
        
        fc_mean_txt1 = torch.cat([mean_txt1, lstm_txt1], 2)
        fm_mean_txt1 = mean_txt1 * lstm_txt1
        fs_mean_txt1 = mean_txt1 - lstm_txt1
        fc_mean_txt2 = torch.cat([mean_txt2, lstm_txt2], 2)
        fm_mean_txt2 = mean_txt2 * lstm_txt2
        fs_mean_txt2 = mean_txt2 - lstm_txt2
        
        fc_mean_txt1, fm_mean_txt1, fs_mean_txt1 = self.compress_mean(fc_mean_txt1, fm_mean_txt1, fs_mean_txt1)
        fc_mean_txt2, fm_mean_txt2, fs_mean_txt2 = self.compress_mean(fc_mean_txt2, fm_mean_txt2, fs_mean_txt2)
        
        # self attention
        self_atten1 = torch.bmm(F.relu(self.attention_self(lstm_txt1.view(size1[0]*size1[1], size1[2]))).view(size1[0], size1[1], self.atten_dim),
                        F.relu(self.attention_self(lstm_txt1.view(size1[0]*size1[1], size1[2]))).view(size1[0], size1[1], self.atten_dim).transpose(1, 2))
        self_atten2 = torch.bmm(F.relu(self.attention_self(lstm_txt2.view(size2[0]*size2[1], size2[2]))).view(size2[0], size2[1], self.atten_dim),
                        F.relu(self.attention_self(lstm_txt2.view(size2[0]*size2[1], size2[2]))).view(size2[0], size2[1], self.atten_dim).transpose(1, 2))
        self_atten1 = F.softmax(self_atten1, dim=2)
        self_atten1 = torch.bmm(self_atten1, lstm_txt1)
        
        self_atten2 = F.softmax(self_atten2, dim=2)
        self_atten2 = torch.bmm(self_atten2, lstm_txt2)
        
        fc_self_txt1 = torch.cat([self_atten1, lstm_txt1], 2)
        fm_self_txt1 = self_atten1 * lstm_txt1
        fs_self_txt1 = self_atten1 - lstm_txt1
        fc_self_txt2 = torch.cat([self_atten2, lstm_txt2], 2)
        fm_self_txt2 = self_atten2 * lstm_txt2
        fs_self_txt2 = self_atten2 - lstm_txt2
        
        fc_self_txt1, fm_self_txt1, fs_self_txt1 = self.compress_self(fc_self_txt1, fm_self_txt1, fs_self_txt1)
        fc_self_txt2, fm_self_txt2, fs_self_txt2 = self.compress_self(fc_self_txt2, fm_self_txt2, fs_self_txt2)
        
        return torch.cat([fc_align_txt1, fm_align_txt1, fs_align_txt1, fc_max_txt1, fm_max_txt1, fs_max_txt1,
                         fc_mean_txt1, fm_mean_txt1, fs_mean_txt1,fc_self_txt1, fm_self_txt1, fs_self_txt1], 2), torch.cat([fc_align_txt2, fm_align_txt2, fs_align_txt2, fc_max_txt2, fm_max_txt2, fs_max_txt2,
                         fc_mean_txt2, fm_mean_txt2, fs_mean_txt2, fc_self_txt2, fm_self_txt2, fs_self_txt2], 2)
                
                
        
class Compression(nn.Module):
    def __init__(self, input_dim, choice):
        super(Compression, self).__init__()
        self.choice = choice
        if choice == 'nn':
            self.linear_concat = nn.Linear(2*input_dim, 1)
            self.linear_mul = nn.Linear(input_dim, 1)
            self.linear_sub = nn.Linear(input_dim, 1)
        elif choice == 'fm':
            pass
        
    def forward(self, concat, mul, sub):
        size = mul.size()
        if self.choice == 'nn':
            out_concat = F.relu(self.linear_concat(concat.view(concat.size(0)*concat.size(1), concat.size(2))))
            out_mul =  F.relu(self.linear_mul(mul.view(mul.size(0)*mul.size(1), mul.size(2))))
            out_sub =  F.relu(self.linear_sub(sub.view(sub.size(0)*sub.size(1), sub.size(2))))
            return out_concat.view(size[0], size[1], 1), out_mul.view(size[0], size[1], 1), out_sub.view(size[0], size[1], 1)
        elif choice == 'sum':
            pass
        

In [8]:
class MCAN(nn.Module):
    def __init__(self, vocab, embed_dim, atten_dim, hidden_dim, deep_layers, pretrain_embed=torch.tensor([]), is_batch_norm=False, is_drop_out=False):
        super(MCAN, self).__init__()
        self.embed_dim = embed_dim
        self.hidden_dim = hidden_dim
        self.atten_dim = atten_dim
        self.deep_layers = deep_layers
        self.is_batch_norm = is_batch_norm
        self.is_drop_out = is_drop_out
        self.word_embed = nn.Embedding(len(vocab), embed_dim, padding_idx=vocab['<PAD>'])
        self.highway_layer = nn.Linear(embed_dim, embed_dim)
        self.highway_layer2 = nn.Linear(4 * hidden_dim * 4, 4 * hidden_dim * 4)
        self.attention_embed = CoAttention(embed_dim, atten_dim)
        if len(pretrain_embed) != 0:
            self.word_embed.weight.data.copy_(pretrain_embed)
#             self.word_embed.weight.requires_grad = False
        
        # lstm encode operator
        self.lstm_embed = nn.LSTM(embed_dim+12, hidden_dim, batch_first=True, bidirectional=True)
        
        # MLP layer
        self.linear_1 = nn.Linear(4 * hidden_dim * 4, deep_layers[0])
        if self.is_batch_norm:
            self.batch_norm_1 = nn.BatchNorm1d(deep_layers[0])
        if self.is_drop_out:
            self.linear_1_dropout = nn.Dropout(0.8)
        for i, h in enumerate(self.deep_layers[1:], 1):
            setattr(self,'linear_'+str(i+1), nn.Linear(self.deep_layers[i-1], self.deep_layers[i]))
            if self.is_batch_norm:
                setattr(self, 'batch_norm_' + str(i + 1), nn.BatchNorm1d(deep_layers[i]))
            if self.is_drop_out:
                setattr(self, 'linear_'+str(i+1)+'_dropout', nn.Dropout(0.8))
        self.deep_out = nn.Linear(deep_layers[-1], 2)
#         self.attention_feature_layer = nn.Linear(STR_MAXLEN, hidden_dim*2)
        self.lstm_fusion = nn.LSTM(2 * hidden_dim * 4, hidden_dim, batch_first=True, bidirectional=True)

    def forward(self, txt1, txt2, x1_mask, x2_mask, hidden=None):
        embed_txt1 = self.word_embed(txt1)
        embed_txt2 = self.word_embed(txt2)
        embed_txt1 = highway_network(self.highway_layer, embed_txt1, 1)
        embed_txt2 = highway_network(self.highway_layer, embed_txt2, 1)
        new_fea_txt1, new_fea_txt2 = self.attention_embed(embed_txt1, embed_txt2)
        embed_txt1 = torch.cat([embed_txt1, new_fea_txt1], 2)
        embed_txt2 = torch.cat([embed_txt2, new_fea_txt2], 2)
        lstm_txt1 = self.lstm_embed(embed_txt1, None)[0]
        lstm_txt2 = self.lstm_embed(embed_txt2, None)[0]
        lstm_txt1 = lstm_txt1 * x1_mask[:, :, None]
        lstm_txt2 = lstm_txt2 * x2_mask[:, :, None]
        lstm_txt1 = lstm_txt1.unsqueeze(1)
        lstm_txt2 = lstm_txt2.unsqueeze(1)
        
        lstm_txt1_max = F.max_pool2d(lstm_txt1, (len(txt1[0]), 1)).squeeze()
        lstm_txt2_max = F.max_pool2d(lstm_txt2, (len(txt2[0]), 1)).squeeze()
#         txt_substract = torch.abs(lstm_txt1 - lstm_txt2)
#         txt_multiply = torch.mul(lstm_txt1, lstm_txt2)
#         output = self.deep_layer_1(torch.cat([lstm_txt1, lstm_txt2, txt_substract, txt_multiply], 1))
        lstm_txt1_mean = F.avg_pool2d(lstm_txt1, (len(txt1[0]), 1)).squeeze()
        lstm_txt2_mean = F.avg_pool2d(lstm_txt2, (len(txt2[0]), 1)).squeeze() 
        
        final_txt1 = torch.cat([lstm_txt1_max, lstm_txt1_mean], 1)
        final_txt2 = torch.cat([lstm_txt2_max, lstm_txt2_mean], 1)
        txt_substract = torch.abs(final_txt1 - final_txt2)
        txt_multiply = torch.mul(final_txt1, final_txt2)
#         lstm_txt1_max = F.max_pool2d(lstm_fusion_txt1, (len(txt1[0]), 1)).squeeze()
#         lstm_txt2_max = F.max_pool2d(lstm_fusion_txt2, (len(txt2[0]), 1)).squeeze() 
        output = highway_network(self.highway_layer2, torch.cat([final_txt1, final_txt2, txt_substract, txt_multiply], 1), 1)
        output = self.linear_1(output)
        if self.is_batch_norm:
            output = self.batch_norm_1(output)
        output = F.relu(output)
        if self.is_drop_out:
            output = self.linear_1_dropout(output)
        for i in range(1, len(self.deep_layers)):
            output = getattr(self, 'linear_' + str(i + 1))(output)
            if self.is_batch_norm:
                output = getattr(self, 'batch_norm_' + str(i + 1))(output)
            output = F.relu(output)
            if self.is_drop_out:
                output = getattr(self, 'linear_' + str(i + 1) + '_dropout')(output)
        output = self.deep_out(output)
        return F.log_softmax(output)

In [9]:
model = MCAN(TEXT.vocab.stoi, EMBED_DIM, ATTEN_DIM, HIDDEN_DIM, DEEP_LAYERS, matrix)
model.to(DEVICE)
criterion = nn.NLLLoss(weight=torch.tensor(CLASS_WEIGHT).float().to(DEVICE))
# parameters = list(filter(lambda p: p.requires_grad, model.parameters()))
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = StepLR(optimizer, step_size=DECAY_STEP, gamma=DECAY_GAMMA)

In [10]:
print_every = 50
best_state = None
max_metric = 0
# model.to(DEVICE)
def predict(model, data_dl, loss_func, device):
    model.eval()
    res_list = []
    label_list = []
    loss = 0
    for text1, text2, label in data_dl:
        text1 = text1.to(DEVICE)
        text2 = text2.to(DEVICE)
        label = label.to(DEVICE)
        x1_mask, x2_mask = get_mask(text1, text2)
        y_pred = model(text1, text2, x1_mask, x2_mask)
        loss += loss_func(y_pred, label).data.cpu()
        y_pred = y_pred.data.max(1)[1].cpu().numpy()
        res_list.extend(y_pred)
        label_list.extend(label.data.cpu().numpy())
    acc = accuracy_score(res_list, label_list)
    Precision = precision_score(res_list, label_list)
    Recall = recall_score(res_list, label_list)
    F1 = f1_score(res_list, label_list)
    
    return loss, (acc, Precision, Recall, F1)
        
def evaluate(model, txt1, txt2, x1_mask, x2_mask, y):
    pred = model(txt1, txt2, x1_mask, x2_mask)
    out_batch = pred.data.max(1)[1].cpu().numpy()
    F1 = f1_score(out_batch, y)
    return F1

def training_termination(valid_result):
    if len(valid_result) >= 4:
        if valid_result[-1] < valid_result[-2] and \
            valid_result[-2] < valid_result[-3] and \
            valid_result[-3] < valid_result[-4]:
            return True
    return False

valid_iter.create_batches()
valid_batch_num = len(list(valid_iter.batches))
print_flush('start train...')
train_iter.create_batches()
batch_num = len(list(train_iter.batches))
print_flush('batch number %d '%batch_num)
valid_result = []
for epoch in range(EPOCHES):
    for param_group in optimizer.param_groups:
        print('learning rate: %.6f'% param_group['lr'])
    epoch_begin = time()
    total_loss = 0.0
    train_iter.init_epoch()
    batch_count = 0
    batch_begin_time = time()
    for text1, text2, label in train_dl:
        model.train()
        text1 = text1.to(DEVICE)
        text2 = text2.to(DEVICE)
        label = label.to(DEVICE)
        x1_mask, x2_mask = get_mask(text1, text2)
        y_pred = model(text1, text2, x1_mask, x2_mask)
        loss = criterion(y_pred, label)
        model.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        batch_count += 1
        if batch_count % print_every == 0:
            metric = evaluate(model.eval(), text1, text2, x1_mask, x2_mask, label)
            print_flush('[%d %d] loss: %.6f metric: %.6f time: %.1f s' %
                  (epoch + 1, batch_count, total_loss / print_every, metric, time() - batch_begin_time))
            total_loss = 0.0
            batch_begin_time = time()
#     scheduler.step()
    print_flush("Evaluating....")
    loss, (acc, Precision, Recall, F1) = predict(model, valid_dl, criterion, DEVICE)
    valid_result.append(F1)
    print_flush('*'*65)
    print_flush('[epoch %d]. loss: %.6f acc: %.6f f1: %.6f time: %.1f s'%(epoch+1, loss/valid_batch_num, acc, F1, time()-epoch_begin))
    print_flush('*'*65)
    if F1 > max_metric:
        best_state = model.state_dict()
        max_metric = F1
        print_flush("save model...")
        torch.save(best_state, '../datasets/models/baseline_LSTM.pth')
    epoch_begin = time()
    if training_termination(valid_result):
        print_flush("early stop at [%d] epoch!" % (epoch+1))
        break

start train...
batch number 337 
learning rate: 0.001000


D:\Anaconda\lib\site-packages\ipykernel_launcher.py:81: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


[1 50] loss: 0.664888 metric: 0.426471 time: 6.2 s
[1 100] loss: 0.581471 metric: 0.523490 time: 5.9 s
[1 150] loss: 0.544226 metric: 0.563107 time: 5.9 s
[1 200] loss: 0.532716 metric: 0.470588 time: 5.8 s
[1 250] loss: 0.523140 metric: 0.537815 time: 6.0 s
[1 300] loss: 0.518676 metric: 0.574257 time: 5.7 s
Evaluating....
*****************************************************************
[epoch 1]. loss: 0.503271 acc: 0.707081 f1: 0.505813 time: 42.1 s
*****************************************************************
save model...
learning rate: 0.001000
[2 50] loss: 0.499459 metric: 0.466019 time: 5.9 s
[2 100] loss: 0.497131 metric: 0.606061 time: 5.9 s
[2 150] loss: 0.476251 metric: 0.634146 time: 5.9 s
[2 200] loss: 0.475933 metric: 0.513514 time: 5.9 s
[2 250] loss: 0.472190 metric: 0.521008 time: 6.0 s
[2 300] loss: 0.471910 metric: 0.607843 time: 5.8 s
Evaluating....
*****************************************************************
[epoch 2]. loss: 0.486889 acc: 0.716046 f1: 0.